In [78]:
//Web scraper for NFT Round 2 data
open System
open System.Collections.Generic
open System.Net.Http
open System.Text
open System.Text.Json
open System.Threading.Tasks
open System.IO

let nftRound2Data = List<_>()

module WebScraper =

    let Scraper (key) : Task =
        async {
            //get the url of the POST request, specifying which data we want scraped
            let url = "https://cnft.tools/toolsapi/v3/project/teddybearclub2"
            //make HTTP requests
            use client = new HttpClient()

            // Headers - mimics a request from  web browser, so its less obvious that its a scraper
            client.DefaultRequestHeaders.Add("User-Agent", "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)")
            
            // Base payload - copy the paylod from the Networks tab - F# record
            let payloadTemplate = 
                {|
                    project = "none"
                    sort = "asc"
                    method = "rarity"
                    page = 1
                    priceOnly = "all"
                    filters = {| |}
                    sliders = 
                        {| 
                            minPrice = 0
                            maxPrice = 0
                            minRank = 0
                            maxRank = 0 
                        |}
                    instantSale = false
                    walletCheck = false
                    stakes = []
                    pageSize = 50
                |}

            let totalPages = 41
            let delayInMilliseconds = 1000

            for page in 1 .. totalPages do
                printf "Fetching page %d..." page
                // Update the page number in the payload - for pagination
                //page increments automatically
                let payload = 
                    {| payloadTemplate with 
                        page = page
                    |}

                //from F# record serialize to JSON and then 
                let jsonPayload = JsonSerializer.Serialize(payload)
                //new HTTP content object saying we are sending json
                let content = new StringContent(jsonPayload, Encoding.UTF8, "application/json")

                try
                    // Send the POST request, async F# syntax 
                    let! response = client.PostAsync(url, content) |> Async.AwaitTask
                    if response.IsSuccessStatusCode then
                        let! responseString = response.Content.ReadAsStringAsync() |> Async.AwaitTask

                        let jsonResponse = JsonSerializer.Deserialize<Dictionary<string, obj>>(responseString)

                        // :? casts result to JsonElement
                        match jsonResponse.TryGetValue(key) with
                        | true, results -> 
                            let resultsJson = results :?> JsonElement
                            if resultsJson.ValueKind = JsonValueKind.Array then
                                for element in resultsJson.EnumerateArray() do
                                    nftRound2Data.Add(element) // Add each element individually
                            else
                                printfn $"Unexpected data format for {key}"
                        | _ -> ()

                    else
                        printfn "Failed to fetch page %d. Status code: %A" page response.StatusCode
                        return()
                with
                    | ex -> 
                        printfn "Error on page %d: %s" page ex.Message
                        return()

                do! Async.Sleep(delayInMilliseconds)

            printf "Data scraping complete" 
        } |> Async.StartAsTask :> Task

WebScraper.Scraper("stats")

Id,2
Exception,<null>
Status,WaitingForActivation
IsCanceled,False
IsCompleted,False
IsCompletedSuccessfully,False
CreationOptions,None
AsyncState,<null>
IsFaulted,False


In [79]:
//Will take a bit of time for the 10 elements to fill up - around 20 secs
let firstTen = nftRound2Data |> Seq.take 10 |> Seq.toList
printfn "%A" firstTen

[{
            "assetName": "TeddyBearsClub869",
            "assetID": "869",
            "name": "Teddy Bears Club #869",
            "price": "x",
            "cnftID": "x",
            "iconurl": "QmYQJ2ZbyNCJYcd8xoWP7oMsRK74RpZcPHRo825nNMZHmW",
            "url": "QmYQJ2ZbyNCJYcd8xoWP7oMsRK74RpZcPHRo825nNMZHmW",
            "Background": "Room",
            "Bear": "Pink Bear",
            "Clothes": "Full Space Suit",
            "Face": "wink 2",
            "Handheld": "Clown Lollipop",
            "Head": "None",
            "Skins": "None",
            "Trait Count": "5",
            "encodedName": "54656464794265617273436c7562383639",
            "buildType": "robot",
            "rarityScore": "841",
            "rarityRank": "1",
            "prices": {},
            "listingDate": "x",
            "ownerStakeKey": "stake1uy67jvetyl2usuhlnlg85t4vfcq07ung5fzp0gd5fs4m4mcke6277"
        };
 {
            "assetName": "TeddyBearsClub909",
            "assetID": "909",
        

In [ ]:
//Getting the information regarding trait counts
open System.Text.Json

type NFTTraits = {
    Background: Map<string, int>
    Bear: Map<string, int>
    Clothes: Map<string, int>
    Face: Map<string, int>
    Handheld: Map<string, int>
    Head: Map<string, int>
    Skins: Map<string, int>
    TraitCount: Map<string, int>
}

let loadNFTs (filePath: string) =
    let jsonContent = File.ReadAllText(filePath)
    JsonSerializer.Deserialize<NFTTraits list>(jsonContent)
    
let nftRound1Data = loadNFTs "./JsonScraper/NFT1Traits.json"
let nftRound2Data = loadNFTs "./JsonScraper/NFT2Traits.json"
let nftRound1Traits = List.head nftRound1Data
let nftRound2Traits = List.head nftRound2Data


In [ ]:
//Score NFT Round 1 and 2 Traits - with larger values for rarer traits
open FSharp.Reflection
let r1TotalNfts = 804.0
let r2TotalNfts = 2035.0

let processNFTTraits (nftTraits: NFTTraits) totalNfts =
    // Use FSharp.Reflection to get all the fields in the record dynamically
    let recordType = typeof<NFTTraits>
    let allTraitScores = 
        FSharp.Reflection.FSharpType.GetRecordFields(recordType)
        |> Array.map (fun field ->
            let fieldValue = field.GetValue(nftTraits) :?> Map<string, int>

            let traitScores = 
                fieldValue |> Map.map (fun key value ->
                    Math.Round(100.0 - (float value / float totalNfts) * 100.0, 2) 
                )

            (field.Name, traitScores)
        )
    allTraitScores

let r1ScoredTraits = processNFTTraits nftRound1Traits r1TotalNfts
let r2ScoredTraits = processNFTTraits nftRound2Traits r2TotalNfts

In [113]:
type nftTotals = { 
    Round1: Map<string, int> 
    Round2: Map<string, int> 
    }

let nftTotalsList = [
    { Round1 = Map.ofList [("Total", 804)]; 
    Round2 = Map.ofList [("Total", 2034)] }
]

nftTotalsList.[0]


{ Round1 = map [("Total", 804)]\n Round2 = map [("Total", 2034)] } Round1 key value Total 804 Round2 key value Total 2034

In [99]:
r2ScoredTraits.[0]

(Background, map [(Aqua, 84.42); (Bamboo Forest, 85.31); (Beach, 86.98); ... ]) Item1 Background Item2 key value Aqua 84.42 Bamboo Forest 85.31 Beach 86.98 Lighthouse 91.2 Pink Evening 86.63 Room 83.49 Space 93.61 Sunflowers 88.35

In [63]:
r2ScoredTraits.[1]

(Bear, map [(Black Bear, 89.04); (Gold Bear, 99.36); (Panda Bear, 85.65); ... ]) Item1 Bear Item2 key value Black Bear 89.04 Gold Bear 99.36 Panda Bear 85.65 Pink Bear 81.67 Polar Bear 83.05 Red Panda 84.23 Teddy Bear 77

In [89]:

let thirdElement = r2ScoredTraits.[2]
let thirdtitle = fst thirdElement
printfn "%s:" thirdtitle
let thirdMap = snd thirdElement
thirdMap |> Map.iter (fun key value ->
    printfn "%s: %f" key value
)


Clothes:
Angel Dress: 98.770000
Archaeologist T-shirt: 98.130000
Balloons: 99.850000
Banana Suit: 99.850000
Basketball  Jersey: 98.770000
Beach Tank Top: 97.940000
Black Denim Jacket: 98.130000
Black Official Suit: 98.180000
Black Official Vest: 98.030000
Blue Baseball Shirt: 98.720000
Blue Dino: 98.870000
Brown Blazer: 98.530000
Businessman Suit: 98.230000
Chef Jacket: 98.230000
Chewie Fur: 99.310000
Colorful Balloons: 99.900000
Cop Uniform: 98.570000
Cowboy Sleeveless Shirt: 98.280000
Diamond Kimono: 99.850000
Firefighter Jacket: 98.770000
Fisherman Clothes: 98.430000
Flowers Shirt: 98.380000
Full Space Suit: 99.660000
Goku T-shirt: 99.510000
Gold Medal: 99.660000
Golden Pajamas: 98.180000
Gorilla Suit: 99.460000
Graffiti Jacket: 98.570000
Green Dino: 98.920000
Green T-shirt: 98.230000
Grey Suit: 98.230000
Harvard Shirt: 98.480000
Hero Uniform: 98.380000
Hot Dog: 99.950000
Inflatable: 98.380000
Jokester: 98.330000
King Cloak: 99.610000
Leia's Dress: 98.130000
Medical Scrubs: 98.13000

In [90]:

let thirdElement = r2ScoredTraits.[3]
let thirdtitle = fst thirdElement
printfn "%s:" thirdtitle
let thirdMap = snd thirdElement
thirdMap |> Map.iter (fun key value ->
    printfn "%s: %f" key value
)


Face:
Confident: 92.630000
Dead: 95.430000
Emotionless: 95.530000
Expressionless: 91.250000
Fake Mustache : 94.350000
Flirty: 92.680000
Happy: 91.250000
In Love: 92.480000
Kissing: 95.680000
Mad: 94.740000
Nervous: 95.580000
Neutral: 96.460000
Painful Smile: 96.950000
Playful: 95.140000
Sleepy: 96.170000
Wink 1: 99.850000
Wink 2: 95.870000
Zipped: 95.920000
unamused: 96.020000
wink: 96.070000
wink 2: 99.950000


In [66]:
r2ScoredTraits.[4]

(Handheld, map [(Black Devil Trident, 97.1); (Broom, 95.33); (Clown Lollipop, 96.66); ... ]) Item1 Handheld Item2 key value Black Devil Trident 97.1 Broom 95.33 Clown Lollipop 96.66 Fishing Rod 97.44 Graffiti Spray 95.14 Honey 99.85 Honey Pot 91.11 King Staff 98.38 Ninja Star 97.44 None 66.54 Pirate sword 95.58 Red Devil Trident 97.59 Robe 93.22 Shovel 94.59 Singer Microphone 96.9 Snowboard 95.82 Strawberry 95.48 Sword 95.82

In [91]:

let thirdElement = r2ScoredTraits.[5]
let thirdtitle = fst thirdElement
printfn "%s:" thirdtitle
let thirdMap = snd thirdElement
thirdMap |> Map.iter (fun key value ->
    printfn "%s: %f" key value
)


Head:
Archaeologist Hat: 98.230000
Arrow: 98.480000
Black Mask: 98.480000
Black Red Hat: 97.490000
Black Round Hat: 98.180000
Black Square Hat: 98.130000
Blue Dino Hood: 98.720000
Blue Panama: 98.380000
Bubbles: 98.820000
Chef Hat: 97.690000
Cop Hat: 98.480000
Crown: 99.800000
Fancy Sunglasses: 99.120000
Firefighter Hat: 98.620000
Firefighter Helmet: 97.940000
Fisherman Panama: 98.130000
Goku Hair: 98.330000
Gorilla Head: 98.130000
Green Dino Hood: 98.430000
Halo: 99.560000
Helmet With Water: 99.850000
Hero Mask: 97.940000
Hussar Hat: 97.940000
Ice Cream: 97.990000
Jokester Hat: 97.640000
Leia Hair: 98.620000
Medical Hat: 98.430000
Military Helmet: 98.670000
Mime Hat: 98.030000
Mohawk: 97.940000
Monocle: 99.800000
Navy Cap: 97.990000
None: 90.960000
Pilot Hat: 98.430000
Pirate: 98.620000
Pirate Bandana: 98.430000
Pirate Eyepatch: 97.990000
Rapper Cap: 97.840000
Red Devil: 98.530000
Red Devil Horns: 97.940000
Relaxing Cucumber Eyes: 98.080000
Santa Hat: 98.230000
Scout Glases: 97.690000

In [68]:
r2ScoredTraits.[6]

(Skins, map [(None, 3.93); (Raggedy Bear, 97.84); (Zombie, 98.23)]) Item1 Skins Item2 key value None 3.93 Raggedy Bear 97.84 Zombie 98.23

In [69]:
r2ScoredTraits.[7]

(TraitCount, map [(4, 94.05); (5, 70.17); (6, 38.92); ... ]) Item1 TraitCount Item2 key value 4 94.05 5 70.17 6 38.92 7 96.86

In [ ]:
//Create code to compute the rank of a given subject
//Code to calculate rewards for each subject
//Merge together the list of Round 1 and 2 NFTs, so that its just one db/csv of all the subjects
// and correspondingrewards

In [76]:
//Code to calculate Marketcap Adjustment to 8,000,000 supply

let sumValues (rewardsList: (string * float) list) =
    rewardsList |> List.sumBy (fun (_, value) -> value)

let marketcapAdjustment rewards = 
    let finalReward = (rewards*8000000)/10000000
    finalReward
